In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_07 import *

## Layerwise Sequential Unit Variance (LSUV)

Getting the MNIST data and a CNN

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
x_train, x_valid = normalize_to(x_train, x_valid)

In [5]:
train_ds = Dataset(x_train, y_train)
valid_ds = Dataset(x_valid, y_valid)

bs = 512
c = y_train.max().item() + 1
loss_func = F.cross_entropy
data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [6]:
mnist_view = view_tfm(1, 28, 28)

In [7]:
cbfs = [Recorder, partial(AvgStatsCallback, accuracy), CudaCallback, partial(BatchTransformXCallback, mnist_view)]

In [8]:
nfs = [8, 16, 32, 64, 64]

In [9]:
class ConvLayer(nn.Module):
    def __init__(self, ni, nf, ks=3, stride=2, sub=0., **kwargs):
        super().__init__()
        self.conv = nn.Conv2d(ni, nf, ks, padding=ks//2, stride=stride, bias=True)
        self.relu = GeneralRelu(sub=sub, **kwargs)
        
    def forward(self, x):
        return self.relu(self.conv(x))
    
    @property
    def bias(self):
        return -self.relu.sub
    
    @bias.setter
    def bias(self, s):
        self.relu.sub = -s
        
    @property
    def weight(self):
        return self.conv.weight

Now we're going to look at the paper [All You Need is a Good Init](https://arxiv.org/pdf/1511.06422.pdf), which introduces *Layer-wise Sequential Unit-Variance* (*LSUV*). We initialize our neural net with the usual technique, then we pass a batch through the model and check the outputs of the linear and convolutional layers. We can then rescale the weights according to the actual variance we observe on the activations, and subtract the mean we observe from the initial bias. That way we will have activations that stay normalized.

We repeat this process until we are satisfied with the mean/variance we observe.

Let's start by looking at a baseline:

In [10]:
learn, run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

In [11]:
run.fit(2, learn)

train: [1.86333109375, tensor(0.3742, device='cuda:0')]
valid: [0.99767978515625, tensor(0.7254, device='cuda:0')]
train: [0.3444769921875, tensor(0.8943, device='cuda:0')]
valid: [0.156250634765625, tensor(0.9520, device='cuda:0')]


Now we try the same thing again with LSUV

In [12]:
learn, run = get_learn_run(nfs, data, 0.6, ConvLayer, cbs=cbfs)

In [13]:
#export
def get_batch(dl, run):
    run.xb, run.yb = next(iter(dl))
    for cb in run.cbs:
        cb.set_runner(run)
    run('begin_batch')
    return run.xb, run.yb

In [14]:
xb, yb = get_batch(data.train_dl, run)

We only want the outputs of convolutional or linear layers. To find them, we need a recursive function. We can use `sum(list, [])` to concatenate the lists the function finds (`sum` applies the + operate between the elements of the list you pass it, beginning with the initial state in the second argument).

In [15]:
#export
def find_modules(m, cond):
    if cond(m): 
        return [m]
    return sum([find_modules(o,cond) for o in m.children()], [])

In [16]:
#export
lin_layers = (nn.Conv1d, nn.Conv2d, nn.Conv3d, nn.Linear, nn.ReLU)

def is_lin_layer(l):
    return isinstance(l, lin_layers)

In [17]:
mods = find_modules(learn.model, lambda o: isinstance(o, ConvLayer))

In [18]:
mods

[ConvLayer(
   (conv): Conv2d(1, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 ), ConvLayer(
   (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
   (relu): GeneralRelu()
 )]

In [19]:
def append_stat(hook, mod, inp, out):
    d = out.data
    hook.mean = d.mean().item()
    hook.std = d.std().item()

In [20]:
mdl = learn.model.cuda()

In [21]:
with Hooks(mods, append_stat) as hooks:
    mdl(xb)
    for hook in hooks:
        print('mean', 'std')
        print(hook.mean, hook.std)

mean std
0.41495972871780396 0.949091911315918
mean std
0.28937381505966187 0.7219604253768921
mean std
0.3263041377067566 0.6011371612548828
mean std
0.3057039976119995 0.5519192814826965
mean std
0.1758633553981781 0.3234204649925232


We first adjust the bias terms to make the means 0, then we adjust the standard deviations to make the stds 1 (with a threshold of 1e-3). The `mdl(xb) is not None` clause is just there to pass `xb` through `mdl` and compute all the activations so that the hooks get updated. 

In [22]:
#export
def lsuv_module(m, xb, eps=1e-3):
    '''
    m - module
    xb - The initial train batch
    '''
    h = Hook(m, append_stat)
    while mdl(xb) is not None and abs(h.mean) > eps:
        m.bias = m.bias - h.mean
    while mdl(xb) is not None and abs(h.std - 1) > eps:
        m.weight.data = m.weight.data / h.std
        
    h.remove()
    return h.mean, h.std

In [23]:
for m in mods:
    print(lsuv_module(m, xb))

(0.022257886826992035, 0.9999998211860657)
(0.1354677826166153, 1.0000001192092896)
(0.12510040402412415, 1.0)
(0.13081035017967224, 1.0)
(0.2776840925216675, 1.0)


In [24]:
%time run.fit(2, learn)

train: [0.5841968359375, tensor(0.8129, device='cuda:0')]
valid: [0.1207434814453125, tensor(0.9637, device='cuda:0')]
train: [0.10576078125, tensor(0.9677, device='cuda:0')]
valid: [0.10011312255859375, tensor(0.9694, device='cuda:0')]
CPU times: user 1.8 s, sys: 14.9 ms, total: 1.81 s
Wall time: 1.4 s


LSUV is particularly useful for more complex and deeper architectures that are hard to initialize to get unit variance at the last layer.

## Export

In [25]:
!python notebook2script.py 07a_lsuv.ipynb

Converted 07a_lsuv.ipynb to exp/nb_07a.py
